# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
import pymongo
import pandas as pd

In [2]:
from pymongo import MongoClient
str_conn='mongodb://localhost:27017'  # str_conn por defecto
cursor = MongoClient(str_conn)

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [3]:
db=cursor.companies

In [4]:
colec=db.collection

In [5]:
res=colec.find().limit(1)

In [6]:
query = {'name': 'Babelgum'}

select = {'name': 1, '_id': 0}

res = colec.find(query, select).limit(10)

df1 = pd.DataFrame(res)
df1

,name
0,Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [7]:
query = {'number_of_employees': {'$gte' : 5000}}

select = {'name': 1, '_id': 0}

res = colec.find(query, select).sort('number_of_employees', -1).limit(20)

df2 = pd.DataFrame(res)
df2

,name
0,Siemens
1,IBM
2,Toyota
3,PayPal
4,Nippon Telegraph and Telephone Corporation
5,Samsung Electronics
6,Accenture
7,Tata Consultancy Services
8,Flextronics International
9,Safeway


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [8]:
query ={ '$and' :
        [{'founded_year': {'$gte' : 2000}},
        {'founded_year': {'$lte' : 2005}}]
}

select = {'name': 1, '_id': 0, 'founded_year' : 1}


res = colec.find(query, select).limit(20)

df3 = pd.DataFrame(res)

df3

,name,founded_year
0,Wetpaint,2005
1,Zoho,2005
2,Digg,2004
3,Facebook,2004
4,Omnidrive,2005
5,StumbleUpon,2002
6,Gizmoz,2003
7,Helio,2005
8,Plaxo,2002
9,Technorati,2002


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [9]:
query = {
    '$and': [
        {'ipo.valuation_amount': {'$gte': 100000000}},
        {'founded_year': {'$lt': 2010}}
    ]
}

select = {'name': 1, '_id': 0, 'ipo.valuation_amount' : 1}

res = colec.find(query, select).limit(20)

df4 = pd.json_normalize(res) 
df4

,name,ipo.valuation_amount
0,Facebook,1.040000e+11
1,Twitter,1.810000e+10
2,Yelp,1.300000e+09
3,LinkedIn,9.310000e+09
4,Amazon,1.000000e+11
5,Brightcove,2.900000e+08
6,KIT digital,2.350000e+08
7,Nielsen,1.600000e+09
8,OpenTable,1.050000e+09
9,ChannelAdvisor,2.870000e+08


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [10]:
query = {
    '$and': [
        {'number_of_employees': {'$lt': 1000}},
        {'founded_year': {'$lt': 2005}}
    ]
}

select = {'name': 1, '_id': 0, 'number_of_employees' : 1}

res = colec.find(query, select).sort('number_of_employees', -1).limit(10)

df5 = pd.DataFrame(res) 
df5

,name,number_of_employees
0,Infinera Corporation,974
1,NorthPoint Communications Group,948
2,888 Holdings,931
3,Forrester Research,903
4,Webmetrics,900
5,SonicWALL,900
6,Cornerstone OnDemand,881
7,Buongiorno,800
8,Cvent,800
9,ZoomInfo,800


### 6. All the companies that don't include the `partners` field.

In [11]:
query = {
    'partners': {'$exists': 1}
}

select = {'name': 1, '_id': 0}

res = colec.find(query, select)

df6 = pd.DataFrame(list(res))
len(df6)

18801

In [12]:
query = {
    'partners': {'$size': 0}
}

select = {'name': 1, '_id': 0}

res = colec.find(query, select)

df6 = pd.DataFrame(list(res))
len(df6)

18647

### 7. All the companies that have a null type of value on the `category_code` field.

In [13]:
query = {
    'category_code': None
}

select = {'name': 1, '_id': 0}

res = colec.find(query, select)

df7 = pd.DataFrame(list(res))
df7

,name
0,Collective
1,Snimmer
2,KoolIM
3,Level9 Media
4,VidKing
...,...
2746,Nellix
2747,Cantimer
2748,cruisecritic
2749,Coloroot


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [14]:
query = {

    'number_of_employees' : {

        '$gte' : 100,
        '$lt' : 1000
    }
}


select = {'name': 1, '_id': 0, 'number_of_employees' : 1}

res = colec.find(query, select)

df8 = pd.DataFrame(list(res))
df8

,name,number_of_employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305
3,LifeLock,644
4,Jajah,110
...,...,...
912,UOL (Universo Online),500
913,NextLabs,100
914,OfficialVirtualDJ,102
915,Willdan Group,385


### 9. Order all the companies by their IPO price in a descending order.

In [15]:
# Assuming you have a MongoDB collection named 'colec'
query = {
    'ipo.valuation_amount': {'$ne': None}
}

select = {'name': 1, '_id': 0, 'ipo.valuation_amount' : 1}


result_cursor = colec.find(query, select).sort('ipo.valuation_amount', -1)

df9 = pd.json_normalize(list(result_cursor))
df9.head(10)

,name,ipo.valuation_amount
0,GREE,1.089600e+11
1,Facebook,1.040000e+11
2,Amazon,1.000000e+11
3,Twitter,1.810000e+10
4,Groupon,1.280000e+10
5,Tencent,1.100000e+10
6,Western Digital,9.430000e+09
7,LinkedIn,9.310000e+09
8,BMC Software,6.000000e+09
9,Rackspace,5.440000e+09


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [16]:
query = {
    'number_of_employees': {'$exists': 1}
}

select = {'name': 1, '_id': 0, 'number_of_employees' : 1}

result_cursor = colec.find(query, select).sort('number_of_employees', -1).limit(10)

df10 = pd.json_normalize(list(result_cursor))

df10

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [17]:
query = {
    'founded_month': {'$gte': 6, '$lte' : 12}
}

select = {'name': 1, 'founded_month' : 1, '_id': 0}

result_cursor = colec.find(query, select).limit(1000)

df11 = pd.DataFrame(list(result_cursor))
df11

,name,founded_month
0,Wetpaint,10
1,Zoho,9
2,Digg,10
3,Omnidrive,11
4,Postini,6
...,...,...
995,Openfilm,11
996,uCubd,9
997,MyGreat,7
998,SquareClock,12


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [18]:
query = {
    'founded_year': {'$lt': 2000},
    'acquisition.price_amount': {'$gt': 1000000}
}

select = {
    'name': 1,
    'acquisition.price_amount': 1,
    '_id': 0
}

res = colec.find(query, select)

df12 = pd.json_normalize(list(res))
df12


,name,acquisition.price_amount
0,Postini,6.250000e+08
1,SideStep,1.800000e+08
2,Recipezaar,2.500000e+07
3,Cyworld,7.140000e+06
4,PayPal,1.500000e+09
...,...,...
219,Savvion,4.900000e+07
220,Inventa Technologies,3.000000e+07
221,Universal Microwave,2.320000e+07
222,Advanced Control Components,1.878000e+07


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [19]:
query = {
    'acquisition.acquired_year': {'$gt': 2000},
}

select = {
    'name': 1,
    'acquisition': 1,
    '_id' : 0
}

res = colec.find(query, select).sort('acquisition.price_amount', -1)

df12 = pd.json_normalize(list(res))
df12

,name,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,T-Mobile,3.900000e+10,USD,None,http://techcrunch.com/2011/03/20/in-the-race-f...,"In The Race For More Spectrum, AT&T Is Acquiri...",2011,3.0,20.0,AT&T,at-t
1,Nextel Communications,3.500000e+10,USD,None,http://www.theage.com.au/news/breaking/sprint-...,Sprint completes Nextel acquisition,2005,8.0,15.0,Sprint Nextel,sprint-nextel
2,Goodrich Corporation,1.840000e+10,USD,None,http://www.masshightech.com/stories/2011/09/19...,UTC shells out $18.4 billion for Goodrich,2011,9.0,22.0,United Technologies,united-technologies
3,BEA Systems,8.500000e+09,USD,None,http://www.oracle.com/corporate/press/2008_jan...,Oracle to Acquire BEA Systems,2008,1.0,16.0,Oracle Corporation,oracle
4,Navteq,8.100000e+09,USD,None,http://www.businessweek.com/stories/2007-10-01...,Press Release,2008,7.0,10.0,Nokia,nokia
...,...,...,...,...,...,...,...,...,...,...,...
2595,ALOT,NaN,USD,stock,http://techcrunch.com/2011/10/17/digital-media...,Digital Media Companies Inuvo And Vertro To Merge,2012,3.0,1.0,Inuvo,inuvo
2596,Nellix,NaN,USD,cash_and_stock,http://investor.endologix.com/releasedetail.cf...,Endologix to Acquire Nellix,2010,10.0,28.0,Endologix,endologix
2597,Celestial Semiconductor,NaN,USD,None,http://www.freshnews.com/news/444859/cavium-ne...,CAVIUM NETWORKS SIGNS DEFINITIVE AGREEMENT TO ...,2011,1.0,31.0,Cavium,cavium-networks
2598,MyChances,NaN,USD,None,http://www.parchment.com/press-releases/new-st...,New Student-Developed Site Helps College Hopef...,2011,4.0,NaN,Parchment,parchment


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [20]:
query = {
    'name' : {'$ne' : None},
    'founded_year' : {'$ne' : None}
}
select = {'name' : 1, 'founded_year' : 1, '_id' : 0}

res = colec.find(query, select).sort('founded_year', -1)
df14 = pd.DataFrame(res)
df14

,name,founded_year
0,Fixya,2013
1,Wamba,2013
2,Advaliant,2013
3,Fluc,2013
4,iBazar,2013
...,...,...
13131,Bachmann Industries,1833
13132,DuPont,1802
13133,Alstrasoft,1800
13134,SmallWorlds,1800


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [21]:
query = {
    'founded_day' : {
        '$gte' : 1,
        '$lte' : 7
    }
}
select = {'name': 1, '_id': 0}

res = colec.find(query, select).sort('acquisition.price_amount', -1).limit(10)
df15 = pd.DataFrame(res)
df15


,name
0,Netscape
1,PayPal
2,Zappos
3,Alibaba
4,Postini
5,Danger
6,Clearwell Systems
7,PrimeSense
8,Amobee
9,Vitrue


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [22]:
query = {

        'number_of_employees' : {'$gt' : 4000},
        'category_code' : 'web'
}
select = {'name' : 1, 'number_of_employees' : 1, '_id' : 0}

res = colec.find(query, select).sort('number_of_employees', 1)
df16 = pd.DataFrame(res)
df16

,name,number_of_employees
0,Expedia,4400
1,AOL,8000
2,Webkinz,8657
3,Rakuten,10000
4,Los Angeles Times Media Group,10000
5,Groupon,10000
6,Yahoo!,13600
7,eBay,15000
8,Experian,15500


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [23]:
query = {

    'acquisition.price_amount' : {'$gt' : 10000000},
    'acquisition.price_currency_code' : 'EUR'
    
}

select = {'name' : 1,'_id' : 0}

res = colec.find(query, select)
df17 = pd.json_normalize(res)
df17

,name
0,ZYB
1,Apertio
2,Greenfield Online
3,Webedia
4,Wayfinder
5,Tuenti Technologies
6,BioMed Central


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [24]:
query = {
        'acquisitiom.acquired_month': {'$lte': 3},
        'acquisitiom.acquired_month': {'$gte': 1},
}

select = {'name' : 1, 'acquisition' : 1, '_id' : 0}

res = colec.find(query, select).limit(10)
df18 = pd.DataFrame(res)
df18

""


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [25]:
query = {
    '$and': [
        {'founded_year': {'$gte': 2000}}, # Fundadas entre 2000 y 2010
        {'founded_year': {'$lte': 2010}},
        {'$nor': [{'acquisition.acquired_year': {'$lt': 2011}}]}, # Filtramos por acquired_year mayor que 2011
        {'founded_year' : {'$ne' : None}}, # Controlar que no entren valores nulos
        {'acquisition.acquired_year' : {'$ne' : None}}
    ]
}

select = {'name': 1, 'founded_year' : 1, 'acquisition.acquired_year' : 1, '_id': 0} 
res = colec.find(query, select).limit(10)
dfbonus1 = pd.json_normalize(list(res))
dfbonus1


,name,founded_year,acquisition.acquired_year
0,Wetpaint,2005,2013
1,Digg,2004,2012
2,Geni,2006,2012
3,Kyte,2006,2011
4,Jingle Networks,2005,2011
5,blogTV,2006,2013
6,delicious,2003,2011
7,Revision3,2005,2012
8,iContact,2003,2012
9,Mashery,2006,2013


### 20. All the companies that have been 'deadpooled' after the third year.

In [26]:
query = {
    '$and': [
        { 'founded_year': {'$ne' : None} },
        { 'deadpooled_year': {'$ne': None} },
        {
            '$expr': {
                '$gt': ['$deadpooled_year', { '$add': ['$founded_year', 3] }]
            }
        }
    ]
}

select = {'name': 1, 'deadpooled_year' : 1, 'founded_year' : 1, '_id': 0}

res = colec.find(query, select)

dfbonus2 = pd.DataFrame(list(res))
dfbonus2

,name,founded_year,deadpooled_year
0,Babelgum,2007,2013
1,Thoof,2006,2013
2,Wesabe,2005,2010
3,Stickam,2006,2013
4,AllPeers,2004,2008
...,...,...,...
432,SpeakSoft,2007,2012
433,Tagito,2008,2012
434,Nordic Windpower,2007,2013
435,Nethra Imaging,2003,2012
